# DATA PRE-PROCESSING

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv("./Datasets/LT.csv")
x = dataset.iloc[1:, 1:5].values
y = dataset.iloc[1:, -1].values

In [3]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
imputer.fit(x[:, :-1])
x[:, :-1] = imputer.transform(x[:, :-1])

In [4]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2)

# LINEAR REGRESSION MULTIVARIABLE


In [5]:
from sklearn.linear_model import LinearRegression
lr_m =  LinearRegression()
lr_m.fit(xtrain,ytrain)
lr_m_pred = lr_m.predict(xtest)

In [6]:
np.set_printoptions(precision=2)
print(np.concatenate((ytest.reshape(len(ytest),1), lr_m.predict(xtest).reshape(len(lr_m.predict(xtest)),1)),1))

[[1360.15 1360.12]
 [1540.   1545.18]
 [1620.1  1614.14]
 ...
 [2837.65 2838.92]
 [1260.45 1259.  ]
 [1044.15 1042.4 ]]


# LINEAR REGRESSION SINGLE VARIABLE

In [7]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(xtrain[:,0].reshape(-1,1), ytrain)

LinearRegression()

In [8]:
lr_pred = lr.predict(xtest[:,0].reshape(-1,1))
print(np.concatenate((ytest.reshape(-1,1), lr_pred.reshape(-1,1)),-1))

[[1360.15 1361.79]
 [1540.   1533.74]
 [1620.1  1616.21]
 ...
 [2837.65 2758.73]
 [1260.45 1275.35]
 [1044.15 1033.23]]


# POLYNOMIAL LINEAR REGRESSION

# SUPPORT VECTOR REGRESSION

In [9]:
from sklearn.svm import SVR
clf_svr= SVR(kernel="linear", C=100, gamma="auto")
clf_svr.fit(xtrain,ytrain)

SVR(C=100, gamma='auto', kernel='linear')

In [10]:
svr_pred = clf_svr.predict(xtest)
print(np.concatenate((ytest.reshape(-1,1), svr_pred.reshape(-1,1)),-1))

[[1360.15 1495.84]
 [1540.   1743.35]
 [1620.1  1804.67]
 ...
 [2837.65 3366.29]
 [1260.45 1350.94]
 [1044.15 1036.54]]


# DECISION TREE REGRESSION

# RANDOM FOREST REGRESSION

In [11]:
from sklearn.ensemble import RandomForestRegressor
rand_reg = RandomForestRegressor(n_estimators=10)
rand_reg.fit(xtrain, ytrain)
rand_pred = rand_reg.predict(xtest)

# Comparing Model R2 score

Linear Regression Multivariable

Linear Regression Simple

Linear Regression Polynomial

Support Vector Regression

Decision Tree Regression

Random Forest Regression

# Results